# ✉️ Messages
  <img src="./assets/LC_Messages.png" width="500">

Messages are the fundamental unit of context for models in LangChain. They represent the input and output of models, carrying both the content and metadata needed to represent the state of a conversation when interacting with an LLM.

## Setup

Load and/or check for needed environmental variables

In [ ]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env(".env")

## Human👨‍💻 and AI 🤖 Messages

In [ ]:
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage

agent = create_agent(
    model="groq:llama-3.1-8b-instant", 
    system_prompt="You are a helpful banker. Answer customer queries with warmth and professionalism.",
)

In [ ]:
human_msg = HumanMessage("Hello, I am looking for a home loan. What are your current interest rates?")

result = agent.invoke({"messages": [human_msg]})

In [ ]:
print(result["messages"][-1].content)

In [ ]:
print(type(result["messages"][-1]))

In [ ]:
for msg in result["messages"]:
    print(f"{msg.type}: {msg.content}\n")

### Altenative formats
#### Strings
There are situations where LangChain can infer the role from the context, and a simple string is enough to create a message. 

In [ ]:
'''agent = create_agent(
    model="groq:llama-3.1-8b-instant",
    system_prompt="You are a terse sports poet.",  # This is a SystemMessage under the hood
)'''

In [ ]:
result = agent.invoke({"messages": "Does the bank offer any special home loan program or reduced rates for salaried employees?"})   # This is a HumanMessage under the hood
print(result["messages"][-1].content)

#### Dictionaries

In [ ]:
result = agent.invoke(
    {"messages": {"role": "user", "content": "How long does it take to clear a cheque?"}}   # This is a HumanMessage under the hood
)
print(result["messages"][-1].content)

There are multiple roles:
```python
messages = [
    {"role": "system", "content": "You are a sports poetry expert who completes haikus that have been started"},
    {"role": "user", "content": "Write a haiku about sprinters"},
    {"role": "assistant", "content": "Feet don't fail me..."}
]
```

## Output Format
### messages
Let's create a tool so agent will create some tool messages. 

In [ ]:
from langchain_core.tools import tool

@tool
def wire_transfer_calculator(destination_type: str, transfer_amount: float) -> float:
    """Calculate standard wire transfer fee based on destination type and amount."""
    if destination_type.lower() == "domestic":
        fee = 15.0 if transfer_amount <= 10000 else 25.0
    elif destination_type.lower() == "international":
        fee = 35.0 if transfer_amount <= 10000 else 50.0
    else:
        fee = 50.0  # Default fee for unknown types
    return fee
    

In [ ]:
agent = create_agent(
    model="groq:llama-3.1-8b-instant",
    tools=[wire_transfer_calculator],
    system_prompt="You are a helpful banker. Answer customer queries with warmth and professionalism.",
)

In [ ]:
result = agent.invoke({"messages": "I want to send an international wire transfer of $8,000 to Canada. What will the fee be?"})

In [ ]:
result["messages"][-1].content

In [ ]:
print(len(result["messages"]))

In [ ]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()

### Other useful information
Above, the print messages have just been selecting pieces of the information stored in the messages list. Let's dig into all the information that is available!

In [ ]:
result

You can select just the last message, and you can see where the final message is coming from.

In [ ]:
result["messages"][-1]

In [ ]:
result["messages"][-1].usage_metadata

In [ ]:
result["messages"][-1].response_metadata

### Try it on your own!
Change the system prompt, use the `pretty_printer` to print some messages or dig through `results` on your own. Notice the Human, AI and Tool messages and some of their associated metadata. Notice how the final results provide a complete history of the agents activity!

In [ ]:
agent = create_agent(
    model="groq:llama-3.1-8b-instant",
    tools=[wire_transfer_calculator],
    system_prompt="Your SYSTEM prompt here",
)

In [ ]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()